<a href="https://colab.research.google.com/github/ParalelaUCM/biciMAD/blob/master/triciclos2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!pip install pyspark

In [0]:
import json
from pyspark import SparkContext
sc = SparkContext()

In [0]:
rdd_usages = {}
def mapper_usages(line):
  data = json.loads(line)
  user = data['user_type']
  user_day = data['user_day_code']
  start = data['idunplug_station']
  end = data['idplug_station']
  date = data['unplug_hourTime']['$date'][0:10]
  hora = data['unplug_hourTime']['$date'][11:19]
  time = data['travel_time']
  age = data['ageRange']
  try:
    track = data['track']
  except:
    track = None
  return {"user_type": user,
          "user_day_code": user_day,
          "start": start,
          "end": end,
          "travel_time": time,
          "date": date,
          "hour": hora,
          "age": age,
          "track": track}

In [14]:
directory = '/content'
rdd_usages['2019'] = sc.emptyRDD()
for filename in os.listdir(directory):
    if filename.endswith(".json"):
      #Nos quedamos con la fecha del dataset en formato YYYYMM
      name = filename.split("_")[0]
      rdd_usages[name] = sc.textFile(os.path.join(directory, filename)).map(mapper_usages)
      rdd_usages['2019'] = rdd_usages['2019'].union(rdd_usages[name])
      #DEBUG starts
      print(name)
      #DEBUG ends
    else:
        continue

201901


In [0]:
rddprima = rdd_usages['2019'].map(lambda x: ((x['user_day_code'],x['user_type']),[x['start'],x['end']]))\
                              .groupByKey().map(lambda x: (x[0],list(x[1])))

In [181]:
rddprima.take(1)

[(('aa4275d4a7ef4a0b7fd1fff2eec15dda5774781fdeea6b9b262caa8af132c429', 1),
  [[30, 100]])]

In [0]:
def adic(ldl):
  d = {}
  for elem in ldl:
    key,value = str(elem[0]),str(elem[1])
    if key in d:
      d[key].append(value)
    else:
      d[key]=[value]
  return d

In [0]:
separador = "-->"
def unomas(dic):
  dk = list(dic.keys())
  for key in dk:
    if not isinstance(dic[key],int): #ya es un ciclo
      for elem in dic[key]:      
        if key==elem:
          dic[key+separador+elem]=1 #[integer] indica ciclo
        else:
          spl = key.split(separador)
          if elem in spl:
            pos = spl.index(elem)
            dic[key+separador+elem]=len(spl)-pos #[integer] indica ciclo
          else:
            try:
              dic[key+separador+elem]=dic[elem]
            except KeyError:
              pass
  return dic

In [0]:
def sigue(dic):
  if len(dic) == len(unomas(dic)):
    return dic
  else:
    return (sigue(unomas(dic)))

In [0]:
def tieneciclos(dic):
  resultado = False
  for key,value in dic.items():
    if isinstance(value,int):
      resultado = True
  return resultado


In [0]:
def dameciclos(dic):
  dic2 = {}
  for key,value in dic.items():
    if isinstance(value,int):
      try:
        dic2[value].append(key)
      except:
        dic2[value]=[key]
  return dic2

In [0]:
rdd = rddprima.mapValues(lambda x: adic(x)).mapValues(lambda x: sigue(x))\
          .filter(lambda x: len(x[1])>0).filter(lambda x: tieneciclos(x[1]))\
          .mapValues(lambda x:dameciclos(x))

In [215]:
rdd.take(10)

[(('e803c8d95160b6a328ef497ef33322937c6b9f2a2df1e3f45ccf6e2529265085', 1),
  {2: ['95-->75-->95',
    '11-->75-->11',
    '75-->95-->75',
    '75-->11-->75',
    '95-->11-->75-->11',
    '95-->75-->11-->75',
    '11-->75-->95-->75'],
   3: ['95-->11-->75-->95', '11-->75-->95-->11', '75-->95-->11-->75']}),
 (('3c3b6732a0b384c78791d3d86dfb44f1141ec5075f4ed6f38d445b74bbd86bf5', 1),
  {2: ['143-->157-->143', '157-->143-->157']}),
 (('58ba8b1ada60486c0ce079ade7fd291da026d029db3526885b1c58e2456b53a2', 1),
  {2: ['67-->47-->67', '47-->67-->47']}),
 (('a985a9c1060233924c92ee1c39d74e07c75a00e0c238054cb8e742eb7cc79885', 1),
  {1: ['149-->149']}),
 (('1356d624f8a8e0371b907e6acfb10257749384479c38984c059b85e2eca46e0c', 1),
  {2: ['79-->115-->79', '115-->79-->115']}),
 (('43c0c98ecb31b9a84119e659dee84c51eec5b84c9f18b94a591adadbf5091a97', 1),
  {1: ['168-->168', '157-->168-->168']}),
 (('f155c1f2a548e1234d24f72d8d1c064cbbf287ffe536fbefa45732f61d9bf9b3', 1),
  {2: ['46-->19-->46', '19-->46-->19']}),
 

In [210]:
#todos los ciclos de longitud m:
m=10
a = rdd.filter(lambda x: m in x[1].keys()).mapValues(lambda x: x[m])
print("ciclos de longitud "+str(m+1)+": ")
a.collect()
#aqui tenemos el problema de que repite los 11 posibles ciclos

ciclos de longitud 11: 


[(('de72307865c05f16a90b242c8ccd218d64c57a116b7a94c0b6cec7312c54ba28', 1),
  ['58-->46-->107-->63-->157-->114-->35-->18-->164-->27-->58',
   '46-->107-->63-->157-->114-->35-->18-->164-->27-->58-->46',
   '107-->63-->157-->114-->35-->18-->164-->27-->58-->46-->107',
   '63-->157-->114-->35-->18-->164-->27-->58-->46-->107-->63',
   '157-->114-->35-->18-->164-->27-->58-->46-->107-->63-->157',
   '114-->35-->18-->164-->27-->58-->46-->107-->63-->157-->114',
   '35-->18-->164-->27-->58-->46-->107-->63-->157-->114-->35',
   '18-->164-->27-->58-->46-->107-->63-->157-->114-->35-->18',
   '164-->27-->58-->46-->107-->63-->157-->114-->35-->18-->164',
   '27-->58-->46-->107-->63-->157-->114-->35-->18-->164-->27',
   '26-->58-->46-->107-->63-->157-->114-->35-->18-->164-->27-->58'])]

In [216]:
rddprima.filter(lambda x: x[0][0]=='e803c8d95160b6a328ef497ef33322937c6b9f2a2df1e3f45ccf6e2529265085').collect()

[(('e803c8d95160b6a328ef497ef33322937c6b9f2a2df1e3f45ccf6e2529265085', 1),
  [[95, 11], [11, 75], [75, 95], [95, 75], [75, 11]])]

In [0]:
qwe = sc.emptyRDD()

In [194]:
qwe.isEmpty()

True

In [197]:
for m in range(20):
  print(m,rdd.filter(lambda x: m in x[1].keys()).mapValues(lambda x: x[m]).isEmpty())

0 True
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 True
9 True
10 False
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True


In [0]:
m=3
rddm = rdd.filter(lambda x: m in x[1].keys()).mapValues(lambda x: x[m])

In [203]:
rddm.take(1)

[(('e803c8d95160b6a328ef497ef33322937c6b9f2a2df1e3f45ccf6e2529265085', 1),
  '75-->95-->11-->75')]

In [206]:
rddm.map(lambda x: (x[0][1],len(x[1]))).take(1)

[(1, 17)]